# Mechanical properties of composites using Micromechanics

In [1]:
%matplotlib inline

import numpy as np
from numpy.linalg import inv
import pandas as pd
import matplotlib.pyplot as plt
from simmit import smartplus as sim
from simmit import identify as iden
import os
import itertools

dir = os.path.dirname(os.path.realpath('__file__'))

## Problem 1:

Consider a 2-phase composite, with an Al matrix containing Al2O3 spherical inhomogeneities, with the following elastic constants for each phase:

$E_{Al} = 70000 \quad \nu_{Al} = 0.33$

$E_{Al2O3} = 370000 \quad \nu_{Al2O3} = 0.22$

The volume fraction of the Alumina phase is 0.1

1. Determine the components of the strain concentration (localization) tensor for the Alumina phase, according to the Mori-Tanaka approximation.
2. Determine the components of the effective elastic stiffness tensor of the composite, using the Mori-Tanaka approximation.
3. Deduce the effective Young modulus and the effective Poisson ratio


### Preliminaries

In [2]:
E_Al = 70000 
nu_Al = 0.33
E_Al2O3 = 370000
nu_Al2O3 = 0.22
## Volume fraction of particles
c = 0.1

L_Al = sim.L_iso(E_Al,nu_Al,'Enu')
M_Al = sim.M_iso(E_Al,nu_Al,'Enu')
L_Al2O3 = sim.L_iso(E_Al2O3,nu_Al2O3,'Enu')

S = sim.Eshelby_sphere(nu_Al)
#S_num = sim.Eshelby(L_Al,1,1,1,50,50)
I = np.eye(6)

### Answer to question 1

In [6]:
T_Al2O3 = inv(I+np.dot(S,np.dot(M_Al,L_Al2O3-L_Al)))
sumT = c*T_Al2O3+(1-c)*I;
invsumT = inv(sumT);

A_Al = np.dot(I,invsumT)
A_Al2O3= np.dot(T_Al2O3,invsumT)

print np.array_str(A_Al2O3, precision=4)

[[ 0.3658  0.033   0.033   0.      0.      0.    ]
 [ 0.033   0.3658  0.033   0.      0.      0.    ]
 [ 0.033   0.033   0.3658  0.      0.      0.    ]
 [ 0.      0.      0.      0.3328  0.      0.    ]
 [ 0.      0.      0.      0.      0.3328  0.    ]
 [ 0.      0.      0.      0.      0.      0.3328]]


### Answer to question 2

In [4]:
L_eff = (1-c)*np.dot(A_Al,L_Al) + c*np.dot(A_Al2O3,L_Al2O3)
print np.array_str(L_eff, precision=2)

[[ 115824.2    54850.04   54850.04       0.         0.         0.  ]
 [  54850.04  115824.2    54850.04       0.         0.         0.  ]
 [  54850.04   54850.04  115824.2        0.         0.         0.  ]
 [      0.         0.         0.     30487.08       0.         0.  ]
 [      0.         0.         0.         0.     30487.08       0.  ]
 [      0.         0.         0.         0.         0.     30487.08]]


### Answer to question 3

In [10]:
d = sim.check_symetries(L_eff)
print d

{'props': array([  8.05695979e+04,   3.21372670e-01]), 'maj_sym': 1, 'umat_type': 'ELISO', 'axis': 0}


## Validation using simmit